# Step 1: Inital db setup

In [1]:
# SOURCE: https://ndres.me/post/best-jupyter-notebook-extensions/
# autoreload: Autoreloads external files without having to restart the notebook. To enable it:

%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Initial - Setup environment vars before testing anything
import os
from sqlalchemy import inspect

# import better_exceptions; better_exceptions.hook()

import sys

from IPython.core.debugger import Tracer  # noqa
from IPython.core import ultratb

sys.excepthook = ultratb.FormattedTB(
    mode="Verbose", color_scheme="Linux", call_pdb=True, ostream=sys.__stdout__
)

os.environ["DEBUG"] = "1"
os.environ["TESTING"] = "0"
os.environ["BETTER_EXCEPTIONS"] = "1"

# os.environ["DATABASE_URL"] = "sqlite:///:memory:"
# os.environ["TEST_DATABASE_URL"] = "sqlite:///:memory:"

os.environ["DATABASE_URL"] = "sqlite:///sensor_test.db"
os.environ["TEST_DATABASE_URL"] = "sqlite:///sensor_test.db"

def debug_dump(obj):
    for attr in dir(obj):
        if hasattr(obj, attr):
            print("obj.%s = %s" % (attr, getattr(obj, attr)))
            
def debug_dump_exclude(obj, exclude=["__builtins__", "__doc__"]):
    for attr in dir(obj):
        if hasattr(obj, attr):
            if attr not in exclude:
                print("obj.%s = %s" % (attr, getattr(obj, attr)))

In [3]:
# Initalize database and create schema etc

import logging

from tenacity import after_log, before_log, retry, stop_after_attempt, wait_fixed

from ultron8.api.db.u_sqlite.session import db_session

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

max_tries = 60 * 5  # 5 minutes
wait_seconds = 1

@retry(
    stop=stop_after_attempt(max_tries),
    wait=wait_fixed(wait_seconds),
    before=before_log(logger, logging.INFO),
    after=after_log(logger, logging.WARN),
)
def init():
    try:
        # Try to create session to check if DB is awake
        db_session.execute("SELECT 1")
    except Exception as e:
        logger.error(e)
        raise e


In [4]:
# Get sqlalchemy classes/objects

from ultron8.api.db.u_sqlite.init_db import init_db
from ultron8.api.db.u_sqlite.session import db_session, engine, Session

# make sure all SQL Alchemy models are imported before initializing DB
# otherwise, SQL Alchemy might fail to initialize properly relationships
# for more details: https://github.com/tiangolo/full-stack-fastapi-postgresql/issues/28
from ultron8.api.db.u_sqlite.base import Base

import pandas as pd

# Import SQLAlchemy data classes

In [5]:
from ultron8.api.db_models.packs import Packs
# from ultron8.api.db_models.action import Action
# from ultron8.api.db_models.guid import Guid
# from ultron8.api.db_models.item import Item

# from ultron8.api.db_models.rule import RuleTypeParameter, RuleType, Rules
from ultron8.api.db_models.sensors import Sensors
# from ultron8.api.db_models.timer import TimeDB
from ultron8.api.db_models.trigger import TriggerTypeDB
# from ultron8.api.db_models.trigger import TriggerTypeDB, Trigger, TriggerInstanceDB
# from ultron8.api.db_models.user import User

In [6]:
# Tables should be created with Alembic migrations
# But if you don't want to use migrations, create
# the tables un-commenting the next line
# 2 - generate database schema
Base.metadata.create_all(bind=engine)

# 3 - create a new session
session = Session()

2019-08-24 16:11:47,581 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


2019-08-24 16:11:47,583 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


2019-08-24 16:11:47,585 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1


...

INFO:sqlalchemy.engine.base.Engine:()


2019-08-24 16:11:47,642 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("guid_tracker")


INFO:sqlalchemy.engine.base.Engine:PRAGMA table_info("guid_tracker")


2019-08-24 16:11:47,643 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [7]:
# Try initializing everything now
logger.info("Initializing service")
init()
logger.info("Service finished initializing")




INFO:__main__:Initializing service
INFO:__main__:Starting call to '__main__.init', this is the 1st time calling it.


2019-08-24 16:11:47,683 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2019-08-24 16:11:47,685 INFO sqlalchemy.engine.base.Engine SELECT 1


INFO:sqlalchemy.engine.base.Engine:SELECT 1


2019-08-24 16:11:47,688 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()
INFO:__main__:Service finished initializing


In [8]:
logger.info("Creating initial data")
init_db(db_session)
logger.info("Initial data created")

INFO:__main__:Creating initial data


2019-08-24 16:11:47,737 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.full_name AS user_full_name, user.email AS user_email, user.hashed_password AS user_hashed_password, user.is_active AS user_is_active, user.is_superuser AS user_is_superuser 
FROM user 
WHERE user.email = ?
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.base.Engine:SELECT user.id AS user_id, user.full_name AS user_full_name, user.email AS user_email, user.hashed_password AS user_hashed_password, user.is_active AS user_is_active, user.is_superuser AS user_is_superuser 
FROM user 
WHERE user.email = ?
 LIMIT ? OFFSET ?


2019-08-24 16:11:47,739 INFO sqlalchemy.engine.base.Engine ('admin@ultron8.com', 1, 0)


INFO:sqlalchemy.engine.base.Engine:('admin@ultron8.com', 1, 0)
INFO:__main__:Initial data created


In [9]:
# debug_dump(Action)

# Step 2: Now That we have a working sqlite session, let's try playing w/ our various db_models classes etc

In [10]:
# Step A. Create a pack ( which has actions, triggers, and sensors associated with it )

# Step 3: Import each of the db_models

In [11]:
# from ultron8.api.db_models.action import Action
# from ultron8.api.db_models.guid import Guid
# from ultron8.api.db_models.item import Item
# from ultron8.api.db_models.packs import Packs
# from ultron8.api.db_models.rule import RuleTypeParameter, RuleType, Rules
# from ultron8.api.db_models.sensors import Sensors
# # from ultron8.api.db_models.timer import TimeDB
# from ultron8.api.db_models.trigger import TriggerType, Trigger, TriggerInstanceDB
# from ultron8.api.db_models.user import User

# Step 4: Ok, let us try inserting, and updating data in the database

# PACKS

In [12]:
##########################################
# packs
##########################################

# p = Packs(name='linux', description='Generic Linux actions', keywords='linux', version='0.1.0', python_versions='3', author='Jarvis', email='info@theblacktonystark.com', contributors='bossjones', files='./tests/fixtures/simple/packs/linux', path="./tests/fixtures/simple/packs/linux", actions=[
#     Action(name="check_loadavg", runner_type="remote-shell-script", description="Check CPU Load Average on a Host", enabled=True, entry_point="checks/check_loadavg.py", parameters='{"period": {"enum": ["1","5","15","all"], "type": "string", "description": "Time period for load avg: 1,5,15 minutes, or \'all\'", "default": "all", "position": 0}}')
# ])

# Create - packs
pack_linux = Packs(
    name="linux",
    description="Generic Linux actions",
    keywords="linux",
    version="0.1.0",
    python_versions="3",
    author="Jarvis",
    email="info@theblacktonystark.com",
    contributors="bossjones",
    files="./tests/fixtures/simple/packs/linux",
    path="./tests/fixtures/simple/packs/linux",
    ref="linux"
)

pack_linux

Packs<name=linux,ref=linux>

In [13]:
# Create - actions

# action_check_loadavg = Action(
#     name="check_loadavg",
#     runner_type="remote-shell-script",
#     description="Check CPU Load Average on a Host",
#     enabled=True,
#     entry_point="checks/check_loadavg.py",
#     parameters='{"period": {"enum": ["1","5","15","all"], "type": "string", "description": "Time period for load avg: 1,5,15 minutes, or \'all\'", "default": "all", "position": 0}}',
#     pack=pack_linux
# )

In [14]:
from ultron8.api.models.trigger import TriggerTypeCreate

trigger_type_dict = {
    "name": "file_watch.line",
    "packs_name": "linux",
    "description": "Trigger which indicates a new line has been detected",
    "parameters_schema": {
        "type": "object",
        "properties": {
            "file_path": {
                "description": "Path to the file to monitor",
                "type": "string",
                "required": True
            }
        },
        "additionalProperties": False
    },
    "payload_schema": {
        "type": "object",
        "properties": {
            "file_path": {
                "type": "string"
            },
            "file_name": {
                "type": "string"
            },
            "line": {
                "type": "string"
            }
        }
    }
}

trigger_type_obj = TriggerTypeCreate(**trigger_type_dict)

In [20]:
debug_dump_exclude(trigger_type_obj)

obj.Config = <class 'ultron8.api.models.trigger.TriggerTypeBase.Config'>
obj.__abstractmethods__ = frozenset()
obj.__annotations__ = {'id': <class 'int'>, 'uid': <class 'str'>, 'name': <class 'str'>, 'packs_name': <class 'str'>, 'parameters_schema': typing.Union[typing.Any, NoneType], 'payload_schema': typing.Union[typing.Any, NoneType], 'metadata_file': typing.Union[str, NoneType]}
obj.__class__ = <class 'ultron8.api.models.trigger.TriggerTypeCreate'>
obj.__config__ = <class 'pydantic.main.Config'>
obj.__delattr__ = <method-wrapper '__delattr__' of TriggerTypeCreate object at 0x11af1fb40>
obj.__dict__ = {}
obj.__dir__ = <bound method BaseModel.__dir__ of <TriggerTypeCreate uid=None name='file_watch.line' ref=None packs_name='linux' description='Trigger which indicates a new line has been detected' parameters_schema={'type': 'object', 'properties': {'file_path': {'description': 'Path to the fil… payload_schema={'type': 'object', 'properties': {'file_path': {'type': 'string'}, 'file_nam

In [21]:
dict(trigger_type_obj)

{'uid': None,
 'name': 'file_watch.line',
 'ref': None,
 'packs_name': 'linux',
 'description': 'Trigger which indicates a new line has been detected',
 'parameters_schema': {'type': 'object',
  'properties': {'file_path': {'description': 'Path to the file to monitor',
    'type': 'string',
    'required': True}},
  'additionalProperties': False},
 'payload_schema': {'type': 'object',
  'properties': {'file_path': {'type': 'string'},
   'file_name': {'type': 'string'},
   'line': {'type': 'string'}}},
 'id': None,
 'metadata_file': None}

In [16]:
# action_check_loadavg


In [17]:
# list all variables of global scope
# SOURCE: https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/
%who

Base	 InteractiveShell	 Packs	 Sensors	 Session	 Tracer	 TriggerTypeCreate	 TriggerTypeDB	 after_log	 
before_log	 db_session	 debug_dump	 debug_dump_exclude	 engine	 init	 init_db	 inspect	 logger	 
logging	 max_tries	 os	 pack_linux	 pd	 retry	 session	 stop_after_attempt	 sys	 
trigger_type_dict	 trigger_type_obj	 ultratb	 wait_fixed	 wait_seconds	 


# SENSORS

In [18]:
# sensors_obj = Sensors(
#         class_name="FileWatchSensor",
#         enabled=True,
#         entry_point="file_watch_sensor.py",
#         description="Sensor which monitors files for new lines",
#         trigger_types=[
#             {
#                 "name": "file_watch.line",
#                 "pack": "linux",
#                 "description": "Trigger which indicates a new line has been detected",
#                 "parameters_schema": {
#                     "type": "object",
#                     "properties": {
#                         "file_path": {
#                             "description": "Path to the file to monitor",
#                             "type": "string",
#                             "required": True
#                         }
#                     },
#                     "additionalProperties": False
#                 },
#                 "payload_schema": {
#                     "type": "object",
#                     "properties": {
#                         "file_path": {
#                             "type": "string"
#                         },
#                         "file_name": {
#                             "type": "string"
#                         },
#                         "line": {
#                             "type": "string"
#                         }
#                     }
#                 }
#             }
#         ],
#         pack=pack_linux
#     )

In [19]:
# sensors_obj